# Conexión a la base de datos

In [1]:
from pathlib import Path

ROOT_DIR = Path.cwd().parent
DB_PATH = ROOT_DIR / 'data' / 'unified.db'

print(f"Database path: {DB_PATH}")

Database path: C:\Users\Usuario\PycharmProjects\data-life-cycle-project-2025\data\unified.db


In [2]:
import sqlite3

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

# Análisis de contaminantes en todos los condados de California

## Lectura de medidas

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display

# --- 1) Medidas de calidad del aire: promedio ANUAL por condado y contaminante ---
# Motivación: la media global por condado puede ocultar variación temporal.
# Aquí agregamos las mediciones por (county, year, pollutant) usando la media anual.

POLLUTANTS_OF_INTEREST = ['pm25', 'pm10', 'no2', 'o3']

# Inspecciona los contaminantes disponibles
pollutants_df = pd.read_sql_query('SELECT pollutant_id, pollutant_name FROM pollutants', con)
pollutants_df['pollutant_name_norm'] = (
    pollutants_df['pollutant_name']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({'n02': 'no2'})
)

available = set(pollutants_df['pollutant_name_norm'])
missing = [p for p in POLLUTANTS_OF_INTEREST if p not in available]
if missing:
    print('⚠️  Aviso: no se encontraron estos contaminantes en la tabla pollutants:', missing)
    print('    Contaminantes disponibles (muestra):', sorted(list(available))[:30])

# Extrae TODAS las mediciones para los contaminantes de interés
query_measurements = """
SELECT
    m.measurement_value,
    m.start_time,
    m.end_time,
    s.sensor_id,
    st.station_id,
    st.county_id,
    c.county_name,
    p.pollutant_id,
    p.pollutant_name
FROM measurements AS m
INNER JOIN sensors AS s
    ON m.sensor_id = s.sensor_id
INNER JOIN stations AS st
    ON s.station_id = st.station_id
INNER JOIN counties AS c
    ON st.county_id = c.county_id
INNER JOIN pollutants AS p
    ON s.pollutant_id = p.pollutant_id
WHERE REPLACE(LOWER(TRIM(p.pollutant_name)), 'n02', 'no2') IN ('pm25','pm10','no2','o3')
"""

air_raw = pd.read_sql_query(query_measurements, con)
air_raw['pollutant_name'] = (
    air_raw['pollutant_name']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({'n02': 'no2'})
)

# Limpieza mínima
air_raw['measurement_value'] = pd.to_numeric(air_raw['measurement_value'], errors='coerce')
air_raw = air_raw.dropna(subset=['county_id', 'measurement_value', 'pollutant_name', 'start_time'])

# Año de la medición (robusto: intenta datetime; si falla, usa los 4 primeros caracteres)
air_raw['start_time'] = pd.to_datetime(air_raw['start_time'], errors='coerce')
if air_raw['start_time'].isna().all():
    # fallback si el formato no es parseable
    air_raw['year'] = air_raw['start_time'].astype(str).str.slice(0, 4)
    air_raw['year'] = pd.to_numeric(air_raw['year'], errors='coerce')
else:
    air_raw['year'] = air_raw['start_time'].dt.year

air_raw = air_raw.dropna(subset=['year'])
air_raw['year'] = air_raw['year'].astype(int)

print('Air raw shape:', air_raw.shape)
air_raw

Air raw shape: (245954, 10)


,measurement_value,start_time,end_time,sensor_id,station_id,county_id,county_name,pollutant_id,pollutant_name,year
0,0.018,2016-03-11 15:00:00+00:00,2016-03-11T16:00:00Z,1111,635,11,Solano,2,o3,2016
1,0.026,2016-03-11 16:00:00+00:00,2016-03-11T17:00:00Z,1111,635,11,Solano,2,o3,2016
2,0.037,2016-03-11 21:00:00+00:00,2016-03-11T22:00:00Z,1111,635,11,Solano,2,o3,2016
3,0.037,2016-03-12 00:00:00+00:00,2016-03-12T01:00:00Z,1111,635,11,Solano,2,o3,2016
4,0.041,2016-03-12 02:00:00+00:00,2016-03-12T03:00:00Z,1111,635,11,Solano,2,o3,2016
...,...,...,...,...,...,...,...,...,...,...
245949,0.045,2016-03-26 20:00:00+00:00,2016-03-26T21:00:00Z,1101,627,10,Yolo,2,o3,2016
245950,0.046,2016-03-26 21:00:00+00:00,2016-03-26T22:00:00Z,1101,627,10,Yolo,2,o3,2016
245951,0.042,2016-03-26 22:00:00+00:00,2016-03-26T23:00:00Z,1101,627,10,Yolo,2,o3,2016
245952,0.041,2016-03-26 23:00:00+00:00,2016-03-27T00:00:00Z,1101,627,10,Yolo,2,o3,2016


In [5]:
# Agregación ANUAL a nivel condado: media por (county, year, pollutant)
air_county_year = (
    air_raw
    .groupby(['county_id', 'county_name', 'year', 'pollutant_name'], as_index=False)
    .agg(
        mean_value=('measurement_value', 'mean'),
        n_measurements=('measurement_value', 'size'),
        n_stations=('station_id', 'nunique'),
    )
)
air_county_year

,county_id,county_name,year,pollutant_name,mean_value,n_measurements,n_stations
0,1,Yuba,2016,pm25,2.085714,105,2
1,1,Yuba,2018,pm25,26.594142,239,4
2,1,Yuba,2020,pm25,55.783548,389,8
3,1,Yuba,2024,pm25,110.336556,900,9
4,2,Santa Cruz,2016,o3,0.019143,7,1
...,...,...,...,...,...,...,...
684,55,Del Norte,2020,pm25,34.467742,124,3
685,55,Del Norte,2021,pm25,5.786765,136,3
686,55,Del Norte,2022,pm25,3.058824,34,2
687,55,Del Norte,2023,pm25,7.991935,124,3


In [6]:
# Pivot para tener una columna por contaminante
air_wide_year = (
    air_county_year
    .pivot(index=['county_id', 'county_name', 'year'], columns='pollutant_name', values='mean_value')
    .reset_index()
)

# Asegura columnas para todos los contaminantes (aunque estén vacías)
for p in POLLUTANTS_OF_INTEREST:
    if p not in air_wide_year.columns:
        air_wide_year[p] = np.nan

print('Air county-year shape:', air_wide_year.shape)
air_wide_year

Air county-year shape: (331, 7)


pollutant_name,county_id,county_name,year,no2,o3,pm10,pm25
0,1,Yuba,2016,NaN,NaN,NaN,2.085714
1,1,Yuba,2018,NaN,NaN,NaN,26.594142
2,1,Yuba,2020,NaN,NaN,NaN,55.783548
3,1,Yuba,2024,NaN,NaN,NaN,110.336556
4,2,Santa Cruz,2016,NaN,0.019143,NaN,4.937198
...,...,...,...,...,...,...,...
326,55,Del Norte,2020,NaN,NaN,NaN,34.467742
327,55,Del Norte,2021,NaN,NaN,NaN,5.786765
328,55,Del Norte,2022,NaN,NaN,NaN,3.058824
329,55,Del Norte,2023,NaN,NaN,NaN,7.991935


## Lectura de datos de prevalencia de asma

In [7]:
import pandas as pd
import numpy as np
from IPython.display import display

# --- 2) Prevalencia de asma por (condado, año, grupo demográfico) ---

asthma_df = pd.read_sql_query(
    """
    SELECT
        ap.id,
        ap.county_id,
        ap.year_from,
        ap.year_to,
        ap.demographic_group_id,
        ap.current_prevalence,
        ap.ci_95_lower,
        ap.ci_95_upper,
        dg.strata,
        dg.age_group,
        dg.age_min,
        dg.age_max
    FROM asthma_prevalence AS ap
    LEFT JOIN demographic_group AS dg
        ON ap.demographic_group_id = dg.id
    """,
    con,
)

# normaliza texto
for col in ['strata', 'age_group']:
    asthma_df[col] = asthma_df[col].astype(str).str.strip()

# Numéricos
asthma_df['current_prevalence'] = pd.to_numeric(asthma_df['current_prevalence'], errors='coerce')
asthma_df['ci_95_lower'] = pd.to_numeric(asthma_df['ci_95_lower'], errors='coerce')
asthma_df['ci_95_upper'] = pd.to_numeric(asthma_df['ci_95_upper'], errors='coerce')

asthma_df['year_from_eff'] = pd.to_numeric(asthma_df['year_from'], errors='coerce')
asthma_df['year_to_eff'] = pd.to_numeric(asthma_df['year_to'], errors='coerce').fillna(asthma_df['year_from_eff'])

# Año representativo para el emparejamiento anual
asthma_df['year'] = asthma_df['year_to_eff']

# Filtra lo esencial
asthma_df = asthma_df.dropna(subset=['county_id', 'demographic_group_id', 'current_prevalence', 'year'])
asthma_df['year'] = asthma_df['year'].astype(int)

# Si existieran múltiples filas para (county, group, year), elige una de forma determinista
asthma_yearly = (
    asthma_df
    .sort_values(['county_id', 'demographic_group_id', 'year', 'year_to_eff', 'year_from_eff'],
                 ascending=[True, True, True, False, False])
    .groupby(['county_id', 'demographic_group_id', 'year'], as_index=False)
    .head(1)
)

# Añade nombre de condado desde counties
counties_df = pd.read_sql_query('SELECT county_id, county_name FROM counties', con)
asthma_yearly = asthma_yearly.merge(counties_df, on='county_id', how='left')

print('Asthma yearly shape:', asthma_yearly.shape)
asthma_yearly

Asthma yearly shape: (1039, 16)


,id,county_id,year_from,year_to,demographic_group_id,current_prevalence,ci_95_lower,ci_95_upper,strata,age_group,age_min,age_max,year_from_eff,year_to_eff,year,county_name
0,58,1,2015,2016,1,14.8,9.0,20.5,Total population,All ages,None,None,2015,2016,2016,Yuba
1,464,1,2017,2018,1,13.8,6.8,20.8,Total population,All ages,None,None,2017,2018,2018,Yuba
2,870,1,2019,2020,1,11.4,7.7,15.2,Total population,All ages,None,None,2019,2020,2020,Yuba
3,1276,1,2021,2022,1,14.1,9.2,19.0,Total population,All ages,None,None,2021,2022,2022,Yuba
4,928,1,2019,2020,2,8.7,2.2,15.2,Child vs. adult,0–17 years,0,17,2019,2020,2020,Yuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,1148,58,2019,2020,6,10.9,6.7,15.2,Age groups,18–64 years,18,64,2019,2020,2020,Sierra
1035,1554,58,2021,2022,6,12.1,1.6,22.6,Age groups,18–64 years,18,64,2021,2022,2022,Sierra
1036,394,58,2015,2016,7,8.7,2.2,15.3,Age groups,65+ years,65,None,2015,2016,2016,Sierra
1037,1206,58,2019,2020,7,7.3,4.1,10.5,Age groups,65+ years,65,None,2019,2020,2020,Sierra


In [8]:

# Resumen de grupos presentes
summary_groups = (
    asthma_yearly
    .groupby(['strata', 'age_group'])
    .size()
    .reset_index(name='n')
    .sort_values('n', ascending=False)
)
display(summary_groups)


,strata,age_group,n
5,Child vs. adult,18+ years,230
6,Total population,All ages,229
1,Age groups,18–64 years,227
3,Age groups,65+ years,193
4,Child vs. adult,0–17 years,79
2,Age groups,5–17 years,70
0,Age groups,0–4 years,11


## Medias anuales

In [9]:
from IPython.display import display

# --- 3) Unión: contaminación anual por condado + prevalencia de asma anual ---
# Resultado: un registro por (county, year, demographic_group)
# Nota: air_wide_year es a nivel (county, year). Al unir con asma, los contaminantes
# se replican por grupo demográfico (como corresponde).

# LEFT JOIN para conservar todos los registros de asma; contaminantes pueden quedar NaN
# si no hay sensores para ese condado/año.
df = asthma_yearly.merge(
    air_wide_year.drop(columns=['county_name'], errors='ignore'),
    on=['county_id', 'year'],
    how='left'
)

# Reordenar columnas
cols = [
    'county_id', 'county_name', 'year',
    'demographic_group_id', 'strata', 'age_group', 'age_min', 'age_max',
    'year_from', 'year_to',
    'current_prevalence', 'ci_95_lower', 'ci_95_upper',
    'pm25', 'pm10', 'no2', 'o3'
]
cols_existing = [c for c in cols if c in df.columns]
df = df[cols_existing + [c for c in df.columns if c not in cols_existing]]

print('Merged dataset shape:', df.shape)
display(df.head(10))

# Completitud por contaminante (county × year × group)
print('\nCompletitud por contaminante (county×year×group):')
for p in ['pm25','pm10','no2','o3']:
    if p in df.columns:
        n = df[p].notna().sum()
        print(f"{p}: {n}/{len(df)} ({100*n/len(df):.1f}%)")

# Completitud por grupo demográfico
coverage = (
    df
    .groupby(['strata','age_group'])
    .agg(
        n_rows=('county_id','size'),
        pm25=('pm25', lambda s: s.notna().sum()),
        pm10=('pm10', lambda s: s.notna().sum()),
        no2 =('no2',  lambda s: s.notna().sum()),
        o3  =('o3',   lambda s: s.notna().sum()),
    )
    .reset_index()
    .sort_values(['strata','age_group'])
)

display(coverage)


Merged dataset shape: (1039, 20)


,county_id,county_name,year,demographic_group_id,strata,age_group,age_min,age_max,year_from,year_to,current_prevalence,ci_95_lower,ci_95_upper,pm25,pm10,no2,o3,id,year_from_eff,year_to_eff
0,1,Yuba,2016,1,Total population,All ages,None,None,2015,2016,14.8,9.0,20.5,2.085714,NaN,NaN,NaN,58,2015,2016
1,1,Yuba,2018,1,Total population,All ages,None,None,2017,2018,13.8,6.8,20.8,26.594142,NaN,NaN,NaN,464,2017,2018
2,1,Yuba,2020,1,Total population,All ages,None,None,2019,2020,11.4,7.7,15.2,55.783548,NaN,NaN,NaN,870,2019,2020
3,1,Yuba,2022,1,Total population,All ages,None,None,2021,2022,14.1,9.2,19.0,NaN,NaN,NaN,NaN,1276,2021,2022
4,1,Yuba,2020,2,Child vs. adult,0–17 years,0,17,2019,2020,8.7,2.2,15.2,55.783548,NaN,NaN,NaN,928,2019,2020
5,1,Yuba,2022,2,Child vs. adult,0–17 years,0,17,2021,2022,12.7,2.4,23.1,NaN,NaN,NaN,NaN,1334,2021,2022
6,1,Yuba,2016,3,Child vs. adult,18+ years,18,None,2015,2016,15.3,8.6,22.0,2.085714,NaN,NaN,NaN,174,2015,2016
7,1,Yuba,2018,3,Child vs. adult,18+ years,18,None,2017,2018,15.9,8.1,23.7,26.594142,NaN,NaN,NaN,580,2017,2018
8,1,Yuba,2020,3,Child vs. adult,18+ years,18,None,2019,2020,12.5,8.2,16.7,55.783548,NaN,NaN,NaN,986,2019,2020
9,1,Yuba,2022,3,Child vs. adult,18+ years,18,None,2021,2022,14.6,9.0,20.2,NaN,NaN,NaN,NaN,1392,2021,2022



Completitud por contaminante (county×year×group):
pm25: 514/1039 (49.5%)
pm10: 265/1039 (25.5%)
no2: 200/1039 (19.2%)
o3: 381/1039 (36.7%)


,strata,age_group,n_rows,pm25,pm10,no2,o3
0,Age groups,0–4 years,11,5,3,3,4
1,Age groups,18–64 years,227,113,55,41,82
2,Age groups,5–17 years,70,32,23,17,29
3,Age groups,65+ years,193,97,48,36,67
4,Child vs. adult,0–17 years,79,36,24,20,32
5,Child vs. adult,18+ years,230,116,56,42,84
6,Total population,All ages,229,115,56,41,83


# Gráficas

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- 4) Gráficas para el informe (2016): barras por condado ordenadas + prevalencia overlay ---
# Requisitos:
# - year = 2016
# - x-axis: counties (ordenadas desc por contaminante)
# - barras: nivel anual del contaminante
# - overlay: prevalencia de asma del grupo demográfico (línea/puntos, eje secundario) EN NEGRO + LEYENDA
#
# Produce: 1 figura por (grupo demográfico × contaminante)

YEAR = 2016
TOP_N = None

pollutants = [
    ("pm25", "PM$_{2.5}$"),
    ("pm10", "PM$_{10}$"),
    ("no2",  "NO$_2$"),
    ("o3",   "O$_3$")
]

# Asegura que existe columna de año
if "year" not in df.columns:
    raise KeyError(
        "No encuentro la columna 'year' en df. "
        "Asegúrate de que el merge anual produce una columna 'year' común."
    )

# Filtra año
plot_df = df[df["year"] == YEAR].copy()

# Limpieza mínima
plot_df["current_prevalence"] = pd.to_numeric(plot_df["current_prevalence"], errors="coerce")
plot_df = plot_df.dropna(subset=["current_prevalence", "county_id"])

for c in ["county_name", "strata", "age_group"]:
    if c not in plot_df.columns:
        raise KeyError(f"Falta la columna '{c}' en df. Revisa el merge con counties/demographic_group.")
    plot_df[c] = plot_df[c].astype(str).str.strip()

fig_dir = Path.cwd() / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

def safe(s: str) -> str:
    return "".join(ch if ch.isalnum() else "_" for ch in str(s).strip())

saved = 0

# Orden estable de grupos
groups = (
    plot_df[["strata", "age_group"]]
    .drop_duplicates()
    .sort_values(["strata", "age_group"])
    .itertuples(index=False, name=None)
)

In [20]:
plot_df

,county_id,county_name,year,demographic_group_id,strata,age_group,age_min,age_max,year_from,year_to,current_prevalence,ci_95_lower,ci_95_upper,pm25,pm10,no2,o3,id,year_from_eff,year_to_eff
0,1,Yuba,2016,1,Total population,All ages,None,None,2015,2016,14.8,9.0,20.5,2.085714,NaN,NaN,NaN,58,2015,2016
6,1,Yuba,2016,3,Child vs. adult,18+ years,18,None,2015,2016,15.3,8.6,22.0,2.085714,NaN,NaN,NaN,174,2015,2016
11,1,Yuba,2016,6,Age groups,18–64 years,18,64,2015,2016,14.3,6.6,22.0,2.085714,NaN,NaN,NaN,348,2015,2016
18,2,Santa Cruz,2016,1,Total population,All ages,None,None,2015,2016,11.3,6.0,16.6,4.937198,NaN,NaN,0.019143,44,2015,2016
22,2,Santa Cruz,2016,3,Child vs. adult,18+ years,18,None,2015,2016,10.4,6.1,14.6,4.937198,NaN,NaN,0.019143,160,2015,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,57,Modoc,2016,7,Age groups,65+ years,65,None,2015,2016,8.7,2.2,15.3,NaN,NaN,NaN,NaN,373,2015,2016
1024,58,Sierra,2016,1,Total population,All ages,None,None,2015,2016,7.2,1.2,13.3,NaN,NaN,NaN,NaN,46,2015,2016
1028,58,Sierra,2016,3,Child vs. adult,18+ years,18,None,2015,2016,10.0,2.4,17.6,NaN,NaN,NaN,NaN,162,2015,2016
1032,58,Sierra,2016,6,Age groups,18–64 years,18,64,2015,2016,10.4,0.2,20.7,NaN,NaN,NaN,NaN,336,2015,2016


In [21]:
for strata, age_group in groups:
    gdf = plot_df[(plot_df["strata"] == strata) & (plot_df["age_group"] == age_group)].copy()

    # Si por alguna razón hay duplicados por county para el mismo grupo/año, promediamos
    # (esto también hace el gráfico estable)
    agg_dict = {"current_prevalence": ("current_prevalence", "mean")}
    for col, _ in pollutants:
        if col in gdf.columns:
            agg_dict[col] = (col, "mean")

    gdf = (
        gdf
        .groupby(["county_id", "county_name"], as_index=False)
        .agg(**agg_dict)
    )

    # Para cada contaminante: barras por condado ordenadas desc + overlay de prevalencia
    for col, label in pollutants:
        if col not in gdf.columns:
            continue

        sub = gdf.dropna(subset=[col, "current_prevalence"]).copy()
        if sub.empty:
            continue

        # Ordenar por contaminante desc (y mantener el mismo orden para prevalencia)
        sub[col] = pd.to_numeric(sub[col], errors="coerce")
        sub["current_prevalence"] = pd.to_numeric(sub["current_prevalence"], errors="coerce")
        sub = sub.dropna(subset=[col, "current_prevalence"])
        sub = sub.sort_values(col, ascending=False)

        # Top N si aplica
        if TOP_N is not None:
            sub = sub.head(TOP_N)

        if len(sub) < 2:
            continue

        # IMPORTANTÍSIMO para evitar "inversiones":
        # Reindexamos explícitamente por el orden ya ordenado (sub.index), y luego construimos arrays.
        sub = sub.reset_index(drop=True)

        counties = sub["county_name"].tolist()
        x = np.arange(len(sub))
        pol = sub[col].to_numpy(dtype=float)
        prev = sub["current_prevalence"].to_numpy(dtype=float)

        # Tamaño dinámico (más ancho si hay muchos condados)
        fig_w = max(14, int(0.35 * len(sub)))
        fig, ax1 = plt.subplots(figsize=(fig_w, 7))

        # Barras contaminante (con label para leyenda)
        bars = ax1.bar(x, pol, label=f"{label} (nivel anual)")
        ax1.set_ylabel(f"Nivel anual ({label})")
        ax1.set_xlabel("Condados (ordenados de mayor a menor contaminación)")
        ax1.set_xticks(x)
        ax1.set_xticklabels(counties, rotation=90, ha="center", fontsize=8)

        # Eje secundario: prevalencia (negro, por encima, con label)
        ax2 = ax1.twinx()
        ax2.set_zorder(ax1.get_zorder() + 1)
        ax2.patch.set_visible(False)

        line, = ax2.plot(
            x, prev,
            marker="o",
            color="black",
            zorder=10,
            label="Prevalencia de asma"
        )
        ax2.set_ylabel("Prevalencia de asma (current_prevalence)")

        # Leyenda combinada (barras + línea)
        h1, l1 = ax1.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()
        ax1.legend(h1 + h2, l1 + l2, loc="upper right")

        # Resumen rápido (Pearson) para la figura
        r_txt = ""
        if len(pol) >= 3 and np.nanstd(pol) > 0 and np.nanstd(prev) > 0:
            r = np.corrcoef(pol, prev)[0, 1]
            r_txt = f" | r={r:.2f}"

        title = f"{label} por condado (año {YEAR}) + prevalencia de asma\n{strata} — {age_group}{r_txt}"
        ax1.set_title(title)

        plt.tight_layout()

        fname = f"ranked_counties_{YEAR}__{safe(label)}__{safe(strata)}__{safe(age_group)}.png"
        out_path = fig_dir / fname
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        saved += 1
        print("Figura guardada en:", out_path)

print(f"Total de figuras guardadas: {saved}")


KeyboardInterrupt: 